# BlueETL core transformations

Intialize dataframes to be used later.

In [1]:
import logging

import numpy as np
import pandas as pd

from blueetl.analysis import Analyzer
from blueetl.constants import *
from blueetl.utils import load_yaml

analysis_config_file = "../tests/data/tmp/analysis_config_01.yaml"

loglevel = logging.WARNING
logformat = "%(levelname)s %(name)s: %(message)s"
logging.basicConfig(format=logformat, level=loglevel)
np.random.seed(0)
analysis_config = load_yaml(analysis_config_file)

a = Analyzer(analysis_config, base_path='..', use_cache=True)
a.extract_repo()
# a.calculate_features()

WARNING blueetl.config.simulations: 'c26abad3-0861-4f7d-863c-a42daffd029f/000' is a directory, using BlueConfig
WARNING blueetl.config.simulations: 'c26abad3-0861-4f7d-863c-a42daffd029f/001' is a directory, using BlueConfig
WARNING blueetl.config.simulations: 'c26abad3-0861-4f7d-863c-a42daffd029f/002' is a directory, using BlueConfig
WARNING blueetl.config.simulations: 'c26abad3-0861-4f7d-863c-a42daffd029f/003' is a directory, using BlueConfig


Not required in general, but set an index to the spikes dataframe to show how to work with multiindexes.

In [2]:
df_mi = a.repo.spikes.df.set_index(["simulation_id", "circuit_id", "neuron_class", "window", "trial"])
df_col = df_mi.reset_index()
display(df_mi)
display(df_col)

time      gid
simulation_id circuit_id neuron_class window trial                   
0             0          L23_EXC      w1     0      2000.200   355828
                                             0      2003.975  3241134
                                             0      2010.875   644392
                                             0      2019.400  3254415
                                             0      2019.950   158267
...                                                      ...      ...
3             0          L6_INH       w3     4        24.000  1389002
                                             4        24.125  2180441
                                             4        24.525  1397134
                                             4        24.575  1724717
                                             4        24.675  1371682

[186948 rows x 2 columns]

,simulation_id,circuit_id,neuron_class,window,trial,time,gid
0,0,0,L23_EXC,w1,0,2000.200,355828
1,0,0,L23_EXC,w1,0,2003.975,3241134
2,0,0,L23_EXC,w1,0,2010.875,644392
3,0,0,L23_EXC,w1,0,2019.400,3254415
4,0,0,L23_EXC,w1,0,2019.950,158267
...,...,...,...,...,...,...,...
186943,3,0,L6_INH,w3,4,24.000,1389002
186944,3,0,L6_INH,w3,4,24.125,2180441
186945,3,0,L6_INH,w3,4,24.525,1397134
186946,3,0,L6_INH,w3,4,24.575,1724717


## Filtering

The framework provides `etl.q` that can be used to filter on both columns and levels of the multiindex.

Multiple keywords can be specified as parameters, and all the conditions are matched in the resulting dataframe.
Each parameter can be:

- a scalar value matched for equality
- a list to match any contained scalar value
- a dict for more complex expressions. The supported operators are: `eq`, `ne`, `le`, `lt`, `ge`, `gt`, `in`

In [3]:
df_mi.etl.q(simulation_id=1, window=['w2', 'w3'], gid={'ge': 356000, 'lt': 357000}, time={'lt': 19.5})

time     gid
simulation_id circuit_id neuron_class window trial                
1             0          L23_EXC      w2     0      19.475  356718
                                      w3     0      19.475  356718
                                             1       8.075  356718
                                             1       8.400  356127

The query would be the same even when the multiindex is reset:

In [4]:
df_col.etl.q(simulation_id=1, window=['w2', 'w3'], gid={'ge': 356000, 'lt': 357000}, time={'lt': 19.5})

,simulation_id,circuit_id,neuron_class,window,trial,time,gid
24398,1,0,L23_EXC,w2,0,19.475,356718
24976,1,0,L23_EXC,w3,0,19.475,356718
25305,1,0,L23_EXC,w3,1,8.075,356718
25311,1,0,L23_EXC,w3,1,8.400,356127


The same result can be obtained with plain Pandas, but it can be more difficult to chain and more verbose, as shown in the example below.

On the other side, the Pandas syntax supports more complex queries that cannot be built with `etl.q`.

In [5]:
df_mi.loc[
    (df_mi.index.get_level_values('simulation_id') == 1)
    & (df_mi.index.get_level_values('window').isin(["w2", "w3"]))
    & (df_mi['gid'] >= 356000) 
    & (df_mi['gid'] < 357000)
    & (df_mi['time'] < 19.5)
]

time     gid
simulation_id circuit_id neuron_class window trial                
1             0          L23_EXC      w2     0      19.475  356718
                                      w3     0      19.475  356718
                                             1       8.075  356718
                                             1       8.400  356127

An alternative in Pandas, using the `query` method, is shown below.
However, this is slower than the previous methods.

In [6]:
df_mi.query("simulation_id==1 and window==['w2', 'w3'] and gid >= 356000 and gid < 357000 and time < 19.5")

time     gid
simulation_id circuit_id neuron_class window trial                
1             0          L23_EXC      w2     0      19.475  356718
                                      w3     0      19.475  356718
                                             1       8.075  356718
                                             1       8.400  356127

## Working with MultiIndexes

In [7]:
df_mi.etl.conditions()

FrozenList(['simulation_id', 'circuit_id', 'neuron_class', 'window', 'trial'])

In [8]:
df_mi.etl.complementary_conditions(['trial', 'simulation_id'])

FrozenList(['circuit_id', 'neuron_class', 'window'])

In [9]:
df_mi.etl.labels()

[Int64Index([0, 1, 2, 3], dtype='int64', name='simulation_id'),
 Int64Index([0], dtype='int64', name='circuit_id'),
 CategoricalIndex(['L23_EXC', 'L23_INH', 'L4_EXC', 'L4_INH', 'L5_EXC', 'L5_INH',
                   'L6_EXC', 'L6_INH'],
                  categories=['L23_EXC', 'L23_INH', 'L4_EXC', 'L4_INH', 'L5_EXC', 'L5_INH', 'L6_EXC', 'L6_INH'], ordered=False, dtype='category', name='neuron_class'),
 CategoricalIndex(['w1', 'w2', 'w3'], categories=['w1', 'w2', 'w3'], ordered=False, dtype='category', name='window'),
 Int64Index([0, 1, 2, 3, 4], dtype='int64', name='trial')]

In [10]:
df_mi.etl.labels_of('window')

CategoricalIndex(['w1', 'w2', 'w3'], categories=['w1', 'w2', 'w3'], ordered=False, dtype='category', name='window')

In [11]:
df_mi.etl.remove_conditions(['window', 'trial'])

time      gid
simulation_id circuit_id neuron_class                   
0             0          L23_EXC       2000.200   355828
                         L23_EXC       2003.975  3241134
                         L23_EXC       2010.875   644392
                         L23_EXC       2019.400  3254415
                         L23_EXC       2019.950   158267
...                                         ...      ...
3             0          L6_INH          24.000  1389002
                         L6_INH          24.125  2180441
                         L6_INH          24.525  1397134
                         L6_INH          24.575  1724717
                         L6_INH          24.675  1371682

[186948 rows x 2 columns]

In [12]:
df_mi.etl.keep_conditions(['window', 'trial'])

time      gid
window trial                   
w1     0      2000.200   355828
       0      2003.975  3241134
       0      2010.875   644392
       0      2019.400  3254415
       0      2019.950   158267
...                ...      ...
w3     4        24.000  1389002
       4        24.125  2180441
       4        24.525  1397134
       4        24.575  1724717
       4        24.675  1371682

[186948 rows x 2 columns]

In [13]:
df_mi.etl.add_conditions(conditions=['cond1', 'cond2'], values=[111, 222])

time  \
cond1 cond2 simulation_id circuit_id neuron_class window trial             
111   222   0             0          L23_EXC      w1     0      2000.200   
                                                         0      2003.975   
                                                         0      2010.875   
                                                         0      2019.400   
                                                         0      2019.950   
...                                                                  ...   
            3             0          L6_INH       w3     4        24.000   
                                                         4        24.125   
                                                         4        24.525   
                                                         4        24.575   
                                                         4        24.675   

                                                                    gid  
cond1 cond2 simulation_id circuit_id neuron_class window trial           
111   222   0             0          L23_EXC      w1     0       355828  
                                                         0      3241134  
                                                         0       644392  
                                                         0      3254415  
                                                         0       158267  
...                                                                 ...  
            3             0          L6_INH       w3     4      1389002  
                                                         4      2180441  
                                                         4      1397134  
                                                         4      1724717  
                                                         4      1371682  

[186948 rows x 2 columns]

## Iteration

In general, iterations can be slow and should be avoided in favour of vectorized operations.

If not possible, the method `etl.iter()` could be used in a similar way to the Pandas `itertuples()`, but it returns a namedtuple also for the index, with only a small performance penalty:

In [14]:
for index, values in df_mi.etl.iter():
    print(index)
    print(values)
    break

Index(simulation_id=0, circuit_id=0, neuron_class='L23_EXC', window='w1', trial=0)
Values(time=2000.2, gid=355828)


For comparison, this is the result from Pandas `itertuples`:

In [15]:
for item in df_mi.itertuples():
    print(item)
    break

Pandas(Index=(0, 0, 'L23_EXC', 'w1', 0), time=2000.2, gid=355828)


Pandas `iterrows` method should be avoided because it's a lot slower than the other methods:

In [16]:
for index, values in df_mi.iterrows():
    print(index)
    print(values)
    break

(0, 0, 'L23_EXC', 'w1', 0)
time      2000.2
gid     355828.0
Name: (0, 0, L23_EXC, w1, 0), dtype: float64


The `iter` method can be used also to iterate over a Series, and in this case the value is just the value of the item:

In [17]:
for index, values in df_mi['time'].etl.iter():
    print(index)
    print(values)
    break

Index(simulation_id=0, circuit_id=0, neuron_class='L23_EXC', window='w1', trial=0)
2000.2


## Grouping

In some cases it can be useful to group by all the conditions, except some of them.

In [18]:
df_mi.etl.groupby_excluding(['neuron_class', 'window', 'trial']).count()

,,time,gid
simulation_id,circuit_id,,
0,0,19712,19712
1,0,55284,55284
2,0,23293,23293
3,0,88659,88659


Remove one or more conditions grouping by the remaining conditions:

In [19]:
from scipy.stats import entropy

def response_entropy(x):
    return entropy(x, base=2) / np.log2(len(x))

df_mi['time'].etl.groupby_excluding(['neuron_class', 'window', 'trial']).apply(response_entropy)

simulation_id  circuit_id
0              0             0.977530
1              0             0.980486
2              0             0.968767
3              0             0.967783
Name: time, dtype: float64

The method `etl.groupby_iter` returns a generator yielding each record of the grouped dataframe as a tuple (key, df).

In general, it's not a replacement of `groupby` because it returns a generator, and not a `DataFrameGroupBy`.

However, it can be used as a replacement for the iteration over `df.groupby`, with these differences:
- the yielded keys are namedtuples, instead of tuples
- the yielded dataframes contain only the columns with varying values, instead of all the columns


Iterating over `df_col`:

In [20]:
for key, tmp_df in df_col.etl.groupby_iter(['simulation_id', 'circuit_id', 'neuron_class', 'window', 'gid']):
    print(key)
    print(tmp_df)
    break

RecordKey(simulation_id=0, circuit_id=0, neuron_class='L23_EXC', window='w1', gid=11359)
     trial      time
629      0  4184.175
909      0  5212.200


For comparison, with Pandas:

In [21]:
for key, tmp_df in df_col.groupby(['simulation_id', 'circuit_id', 'neuron_class', 'window', 'gid']):
    print(key)
    print(tmp_df)
    break

(0, 0, 'L23_EXC', 'w1', 11359)
     simulation_id  circuit_id neuron_class window  trial      time    gid
629              0           0      L23_EXC     w1      0  4184.175  11359
909              0           0      L23_EXC     w1      0  5212.200  11359


Iterating over `df_mi`:

In [22]:
for key, tmp_df in df_mi.etl.grouped_by(['simulation_id', 'circuit_id', 'neuron_class', 'window', 'gid']):
    print(key)
    print(tmp_df)
    break

RecordKey(simulation_id=0, circuit_id=0, neuron_class='L23_EXC', window='w1', gid=11359)
                                                        time
simulation_id circuit_id neuron_class window trial          
0             0          L23_EXC      w1     0      4184.175
                                             0      5212.200


For comparison, with Pandas:

In [23]:
for key, tmp_df in df_mi.groupby(['simulation_id', 'circuit_id', 'neuron_class', 'window', 'gid']):
    print(key)
    print(tmp_df)
    break

(0, 0, 'L23_EXC', 'w1', 11359)
                                                        time    gid
simulation_id circuit_id neuron_class window trial                 
0             0          L23_EXC      w1     0      4184.175  11359
                                             0      5212.200  11359


## Multiprocess

With `etl.groupby_run_parallel()` it's possible to group the dataframe and run a function in parallel processes.

The function should accept (key, df) and it can return anything.

This method should be used only when the function is expensive and it's worth to execute it in a separate subprocess.

If the function isn't expensive, iterating over `etl.grouped_by()` and calling the function is probably faster.

In [24]:
def func(key, df):
    return key, len(df)

df_mi.etl.groupby_run_parallel(['simulation_id', 'circuit_id', 'neuron_class'], func=func)[:3]

[(RecordKey(simulation_id=0, circuit_id=0, neuron_class='L23_EXC'), 2324),
 (RecordKey(simulation_id=0, circuit_id=0, neuron_class='L23_INH'), 247),
 (RecordKey(simulation_id=0, circuit_id=0, neuron_class='L4_EXC'), 3354)]

If the function returns a DataFrame and all the DataFrames should be concatenated together to create a single DataFrame, it's possible to use `etl.groupby_apply_parallel()`.

This is just a shortcut to calling `etl.groupby_run_parallel()` and concatenate the reuslts with `pd.concat()`. Note that it should be considered still experimental and subject to changes.

In [25]:
def func(key, df):
    return pd.DataFrame(
        data={"mean": [df['time'].mean()]},
        index=pd.MultiIndex.from_tuples([key], names=key._fields)
    )

df_col.etl.groupby_apply_parallel(['simulation_id', 'circuit_id', 'window'], func=func)

mean
simulation_id circuit_id window             
0             0          w1      4498.261653
                         w2        33.761481
                         w3        11.607634
1             0          w1      4499.436633
                         w2        38.079973
                         w3        12.027419
2             0          w1      4501.052279
                         w2        33.003820
                         w3        11.950877
3             0          w1      4502.473920
                         w2        46.753784
                         w3        12.903279

The previous one was just an example. To get the same result, it would be more efficient:

In [26]:
df_col.groupby(['simulation_id', 'circuit_id', 'window'])[['time']].mean()

time
simulation_id circuit_id window             
0             0          w1      4498.261653
                         w2        33.761481
                         w3        11.607634
1             0          w1      4499.436633
                         w2        38.079973
                         w3        12.027419
2             0          w1      4501.052279
                         w2        33.003820
                         w3        11.950877
3             0          w1      4502.473920
                         w2        46.753784
                         w3        12.903279